In [1]:
# Импорт нужной для работы библиотеки
import win32com.client
rastr = win32com.client.Dispatch('Astra.Rastr')

In [3]:
import sys

# Если данная ячейка завершается ошибкой,
# значит вы используете не 32-битную версию Python
assert sys.maxsize < 2**32, \
    'Incorrect Python architecture. We should use Python 32-bit'

In [4]:
# Загружаем режим
rastr.Load(1, 'regime.rg2', '')
result = rastr.rgm('p')
print(result)

0


In [15]:
# Парсим json с сечением
import json

with open('flowgate.json', "r") as flowgate:
    flowgate_data = json.load(flowgate)

print(flowgate_data)

#for line in flowgate_data:
#    print(flowgate_data[line]['ip'])

{'line_1': {'ip': 17, 'iq': 16, 'np': 0}, 'line_2': {'ip': 6, 'iq': 11, 'np': 0}, 'line_3': {'ip': 4, 'iq': 14, 'np': 0}}


In [6]:
# Парсим json с отключениями
import json

faults = open('faults.json', "r")
faults_data = json.load(faults)

print(faults_data)

#for lines in faults_data:
#    print(faults_data[lines]['ip'])

{'outage_of_6_11': {'ip': 6, 'iq': 11, 'np': 0, 'sta': 1}, 'outage_of_4_14': {'ip': 4, 'iq': 14, 'np': 0, 'sta': 1}}


In [16]:
# Парсим csv с траекторией
import csv

trajectory_data = []

vector = open('vector.csv', newline = '')
vector_data = csv.DictReader(vector)

for row in vector_data:
    trajectory_data.append(row)

#print(trajectory_data)
print(len(trajectory_data))

#for node in trajectory_data:
#   print(node)

22


In [8]:
#Словарик узлов
node_data = {}
i = 0
for node in trajectory_data:
    if node['node'] not in node_data:
        node_data[node['node']] = i
    i = i + 1
print(node_data)

{'23': 0, '24': 1, '29': 2, '21': 3, '27': 4, '3': 5, '16': 6, '8': 7, '7': 8, '15': 9, '4': 10, '20': 11, '25': 12, '1': 13, '2': 14, '5': 17, '6': 18, '9': 21}


In [9]:
# Формируем и загружаем шаблоны
rastr.Save('sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')

rastr.Save('traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

In [10]:
#Заполняем sech.sch
FlowGate = rastr.Tables('sechen')
GroupLine = rastr.Tables('grline')

FlowGate.DelRowS()
GroupLine.DelRows()

FlowGate.AddRow()
FlowGate.Cols('name').SetZ(0, 'new')
FlowGate.Cols('ns').SetZ(0, 1)
FlowGate.Cols('sta').SetZ(0, 1)

i = 0

for line in flowgate_data:
    GroupLine.AddRow()
    GroupLine.Cols('ns').SetZ(i, 1)

    start_node = flowgate_data[line]['ip']
    end_node = flowgate_data[line]['iq']

    GroupLine.Cols('ip').SetZ(i, start_node)
    GroupLine.Cols('iq').SetZ(i, end_node)
    
    i = i + 1

rastr.Save('sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')

In [11]:
#Заполняем traj.ut2
Trajectory = rastr.Tables('ut_node')

i = 0

Trajectory.DelRowS()

for node in trajectory_data:
    node_type = node['variable']
    node_number = node['node']
    power_change = float(node['value'])
    power_tg = node['tg']



    Trajectory.AddRow()
    Trajectory.Cols('ny').SetZ(i, node_number)
    Trajectory.Cols(node_type).SetZ(i, power_change)
    
    #if node_type == 'pn':
    #    Trajectory.Cols('pn').SetZ(i, power_change)
    #if node_type == 'pg':
    #    Trajectory.Cols('pg').SetZ(i, power_change)
    if Trajectory.Cols('tg').Z(i) == 0:
        Trajectory.Cols('tg').SetZ(i, power_tg)
    
    i = i + 1

rastr.Save('traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')


In [12]:
# Тестовое утяжеление
rastr.Load(1, 'regime.rg2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/режим.rg2')
rastr.Load(1, 'sech.sch', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/сечения.sch')
rastr.Load(1, 'traj.ut2', 'C:/Users/mishk/Documents/RastrWin3/SHABLON/траектория утяжеления.ut2')

result = rastr.rgm('p')
print(result)

0


In [13]:
# Параметры утяжеления
PowerFlowControl = rastr.Tables('ut_common')
PowerFlowControl.Cols('iter').SetZ(0, 200)
PowerFlowControl.Cols('enable_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_v_contr').SetZ(0, 1)
PowerFlowControl.Cols('dis_i_contr').SetZ(0, 1)

In [14]:
#Пробное утяжеление
if rastr.ut_utr('i') > 0:
    rastr.ut_utr('')

maxPowerFlow = rastr.Tables('sechen').Cols('psech').Z(0)
print(maxPowerFlow)

-3160.550806388896
